In [21]:
#%pip install simpletransformers
model_files_output_folder="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\models"
input_data_file          ="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\data\\input_data.csv"
output_data_file         ="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\data\\output_data.csv"
import pandas as pd
from simpletransformers.classification import ClassificationModel
from utils.text_processing import text_processing

In [2]:
#read model from file
model2 = ClassificationModel("roberta", model_files_output_folder,use_cuda=False)

d:\Sola\DEV\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
#Prepare file for evaluation
df = pd.read_csv(input_data_file, header=0)
data = {}

df = pd.DataFrame(df[['uid','source_id'
                      ,'source','article_date'
                      ,'content','url','count'
                      ,'img_link','entity'
                      ,'author','coin']])
#df = df.rename({'label': 'labels'}, axis=1)


df['content'] = df.apply(lambda x: text_processing(x.content,                     
                lower=True, 
                remove_url=True, 
                remove_punctuation=True, 
                remove_stopwords=False, 
                replace_entity=True, 
                replace_hash=True,
                split_alphanumeric=False,
                lemmatize=True,
                stem=True), axis=1)

print("Check null: ", df.isnull().sum().sum())
display(df.head(3))

pred, raw_outputs = model2.predict(df['content'].tolist())


Check null:  240


,uid,source_id,source,article_date,content,url,count,img_link,entity,author,coin
0,1f6f9ca3-ca10-4f39-87ef-bc25ba432ac4,NaN,Cointelegraph,2024-01-03,the nba star appear in a june 2022 commerci sa...,https://cointelegraph.com/news/shaquille-o-nea...,1,NaN,regulaton,NaN,NaN
1,d237c9a7-13b6-4602-8e14-27df1868b4b1,NaN,Cointelegraph,2024-01-15,at a recent event tom mutton the bank of engla...,https://cointelegraph.com/news/boe-fintech-hea...,1,NaN,regulaton,NaN,NaN
2,9f0a6cca-8638-4411-8ee5-5447cb7f0f32,NaN,Cointelegraph,2024-01-22,ishan wahi must surrend by june 21 to serv his...,https://cointelegraph.com/news/former-coinbase...,1,NaN,regulaton,NaN,NaN


100%|██████████| 8/8 [00:14<00:00,  1.84s/it]


In [19]:
from scipy.special import softmax
df['ground_truth_risk'] = pred
probabilities = softmax(raw_outputs, axis=1)
probabilities_1 = [x[1] for x in probabilities]
predicted = [x[1]*100.0 for x in probabilities]

df['probability_risk'] = probabilities_1
df['predicted'] = predicted

In [22]:
df.to_csv(output_data_file, index=False)
display(df.head(9))

,uid,source_id,source,article_date,content,url,count,img_link,entity,author,coin,ground_truth_risk,probability_risk,predicted
0,1f6f9ca3-ca10-4f39-87ef-bc25ba432ac4,NaN,Cointelegraph,2024-01-03,the nba star appear in a june 2022 commerci sa...,https://cointelegraph.com/news/shaquille-o-nea...,1,NaN,regulaton,NaN,NaN,1,0.777141,77.714051
1,d237c9a7-13b6-4602-8e14-27df1868b4b1,NaN,Cointelegraph,2024-01-15,at a recent event tom mutton the bank of engla...,https://cointelegraph.com/news/boe-fintech-hea...,1,NaN,regulaton,NaN,NaN,0,0.261247,26.124737
2,9f0a6cca-8638-4411-8ee5-5447cb7f0f32,NaN,Cointelegraph,2024-01-22,ishan wahi must surrend by june 21 to serv his...,https://cointelegraph.com/news/former-coinbase...,1,NaN,regulaton,NaN,NaN,0,0.007493,0.749311
3,3653e578-d857-4b2c-a4f8-53864d1c525f,NaN,Cointelegraph,2024-01-08,the lion share of the fintech hold cryptocurr ...,https://cointelegraph.com/news/paypal-s-crypto...,1,NaN,regulaton,NaN,NaN,0,0.422213,42.221331
4,93401c1e-80f5-4a25-956a-2f61d06a7a8b,NaN,Cointelegraph,2024-01-29,coinbas chief legal offic request revis to the...,https://cointelegraph.com/news/coinbase-legal-...,1,NaN,regulaton,NaN,NaN,0,0.194210,19.421000
5,2c7a5d65-7b8e-432c-93f5-6747c2d62d6a,NaN,Cointelegraph,2024-02-05,estonia money launder regul highlight a number...,https://cointelegraph.com/news/estonia-squeeze...,1,NaN,regulaton,NaN,NaN,1,0.923341,92.334071
6,85b7862b-41a6-4c1f-9657-3427e8b15114,NaN,Cointelegraph,2024-01-11,industri repres have cast doubt on the legal a...,https://cointelegraph.com/news/war-on-crypto-n...,1,NaN,regulaton,NaN,NaN,0,0.013634,1.363427
7,182d306c-7a29-462c-8a5f-b73b182e546a,NaN,Cointelegraph,2024-02-12,lawyer repres sam bankman fri want all but thr...,https://cointelegraph.com/news/ftx-founder-sam...,1,NaN,regulaton,NaN,NaN,1,0.876314,87.631408
8,4825914a-e390-46a3-9275-1c1614775577,NaN,Cointelegraph,2024-01-18,argentina central bank ban payment provid from...,https://cointelegraph.com/news/argentina-says-...,1,NaN,regulaton,NaN,NaN,0,0.270251,27.025064
